In [46]:
import unicode

unicode.join_jamos("ㅅㅞㅀㅅㅞㅀ")
# '쉟쉟'

'쉟쉟'

In [52]:
from rapidfuzz import process, fuzz
import unicodedata
import difflib
import unicode

# 한영 자판 변환 딕셔너리
eng_to_kor = {
    'r': 'ㄱ', 'R': 'ㄲ', 's': 'ㄴ', 'e': 'ㄷ', 'E': 'ㄸ', 'f': 'ㄹ', 'a': 'ㅁ', 'q': 'ㅂ', 'Q': 'ㅃ', 't': 'ㅅ',
    'T': 'ㅆ', 'd': 'ㅇ', 'w': 'ㅈ', 'W': 'ㅉ', 'c': 'ㅊ', 'z': 'ㅋ', 'x': 'ㅌ', 'v': 'ㅍ', 'g': 'ㅎ',
    'k': 'ㅏ', 'o': 'ㅐ', 'i': 'ㅑ', 'O': 'ㅒ', 'j': 'ㅓ', 'p': 'ㅔ', 'u': 'ㅕ', 'P': 'ㅖ', 'h': 'ㅗ', 
    'y': 'ㅛ', 'n': 'ㅜ', 'b': 'ㅠ', 'm': 'ㅡ', 'l': 'ㅣ'
}

# 초성 리스트
CHOSUNG_LIST = [
    'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'
]

HANGUL_BEGIN_UNICODE = 44032
HANGUL_END_UNICODE = 55203
CHOSUNG_BASE = 588

# 초성 추출 함수
def get_chosung(text):
    result = []
    for char in text:
        if HANGUL_BEGIN_UNICODE <= ord(char) <= HANGUL_END_UNICODE:
            chosung_index = (ord(char) - HANGUL_BEGIN_UNICODE) // CHOSUNG_BASE
            result.append(CHOSUNG_LIST[chosung_index])
        else:
            result.append(char)
    return ''.join(result)

# 영어 입력을 한글로 변환하는 함수
def convert_eng_to_kor(eng_text):
    result = []
    for char in eng_text:
        if char in eng_to_kor:
            result.append(eng_to_kor[char])
        else:
            result.append(char)
    return ''.join(result)

# 입력된 텍스트가 초성인지 확인하는 함수
def is_chosung(text):
    return all(char in CHOSUNG_LIST for char in text)

# 상병명 데이터 리스트 (생략된 데이터 포함)
disease_data = [
    "부분무치증",
    "완전무치증",
    "전치 부위 과잉치",
    "소구치 부위 과잉치",
    "대구치 부위 과잉치",
    "상세불명의 과잉치",
    "왜소치",
    "유착",
    "치아의 크기와 형태의 기타 및 상세불명의 이상",
    "법랑질형성저하 Enamel hypoplasia",
    "절렬 (치아 만곡)",
    "불완전 에나멜 형성증 Amelogenesis imperfecta",
    "불완전 상아질 형성증 Dentinogenesis imperfecta",
    "잔존 [지속성][탈락성] 유치",
    "치아맹출의 기타 명시된 장애",
    "치아맹출의 상세불명 장애",
    "상악 절치의 매복",
    "하악 절치의 매복",
    "상악 견치의 매복",
    "하악 견치의 매복",
    "상악 소구치의 매복",
    "하악 소구치의 매복",
    "상악 제3대구치의 매복",
    "하악 제3대구치의 매복",
    "과잉 매복치",
    "상세불명의 매복치",
    "가역적 치수염",
    "비가역적 치수염",
    "상세불명의 치수염",
    "치아뿌리낭 (근단 및 외측)",
    "치아뿌리낭 (잔류성)",
    "치아뿌리낭 (염증성)",
    "치아뿌리낭 (상세불명)",
    "급성 연쇄알균 치은 구내염",
    "기타 명시된 급성 치은염",
    "상세불명의 급성 치은염",
    "만성 단순 변연부 치은염",
    "만성 증식성 치은염",
    "만성 궤양성 치은염",
    "만성 박리성 치은염",
    "기타 명시된 만성 치은염",
    "상세불명의 만성 치은염",
    "동이 없는 잇몸 기원의 치주농양",
    "동이 있는 잇몸 기원의 치주농양",
    "급성 치관주위염",
    "기타 명시된 급성 치주염",
    "상세불명의 급성 치주염",
    "만성 단순 치주염",
    "만성 복합 치주염",
    "만성 치관주위염",
    "기타 명시된 만성 치주염",
    "상세불명의 만성 치주염",
    "국소적 치은퇴축",
    "전반적 치은퇴축",
    "치은섬유종증 (gingival fibromatosis)",
    "기타 명시된 치은비대",
    "상세불명의 치은비대",
    "외상성 교합에 의한 (치조융기 병변)",
    "자극성 증식증 [의치성 증식증]",
    "위치이상을 동반한 매몰치 또는 매복치",
    "턱관절내장증",
    "턱관절잡음",
    "턱관절의 재발성 탈구 및 아탈구",
    "달리 분류되지 않은 턱관절의 통증",
    "달리 분류되지 않은 턱관절의 경직",
    "턱관절의 퇴행성 관절병",
    "저작근장애",
    "기타 명시된 턱관절장애",
    "상세불명의 턱관절장애",
    "불규칙치조돌기",
    "치아 및 지지구조의 기타 명시된 장애",
    "급성 타액선염",
    "만성 타액선염",
    "볼의 열린상처",
    "상악부의 열린상처",
    "하악부의 열린상처",
    "볼 및 측두하악부 영역의 기타 및 여러 부위의 열린상처",
    "입술의 열린 상처",
    "볼점막의 열린 상처",
    "잇몸(치조돌기)의 열린 상처",
    "혀와 입바닥의 열린 상처",
    "구개의 열린 상처",
    "법랑질만의 파절",
    "치수침범이 없는 치관의 파절",
    "치수침범이 있는 치관의 파절",
    "치근의 파절",
    "치근을 포함한 치관의 파절",
    "치아의 다발성 파절",
    "치아의 상세불명 파절",
    "치아의 아탈구 (측방 탈구)",
    "치아의 함입 또는 탈출",
    "완전 탈구",
    "치아의 박리 (완전탈구)",
    "기타 및 상세불명의 치아 탈구",
]

# 초성 검색 시 초성이 상병명 초성에 포함되는지 확인하여 검색
def search_with_chosung_inclusion(data, query):
    chosung_query = get_chosung(query)  # 입력된 검색어의 초성 추출
    # 데이터의 초성 추출 후, 초성 문자열에 포함되는지 확인
    matches = [item for item in data if chosung_query in get_chosung(item)]
    return matches

def search_with_partial_and_correction(data, query, threshold=50):
    # 1. 부분 일치를 먼저 확인하고, 유사도가 높은 상병명 선택
    matches = process.extract(query, data, scorer=fuzz.partial_ratio, limit=20)
    return [match[0] for match in matches if match[1] >= threshold]

# 검색 실행 함수
def search_with_options(data, query):
    # 초성만 입력된 경우
    if is_chosung(query):
        print("초성 검색을 실행합니다.")
        return search_with_chosung_inclusion(data, query)
    
    # 영어로 입력된 경우 한글로 변환
    if all(char.isalpha() and char.islower() for char in query):
        print("영어 입력을 한글로 변환합니다.")
        query = unicode.join_jamos(convert_eng_to_kor(query))
        print(query)
    
    # 부분 일치 및 오타 보정
    return search_with_partial_and_correction(data, query)

# 검색어
search_query = "만서ㅇ"

# 검색 실행
result = search_with_options(disease_data, search_query)

# 결과 출력
print("검색어:", search_query)
print("검색 결과:", result)


검색어: 만서ㅇ
검색 결과: ['만성 단순 변연부 치은염', '만성 증식성 치은염', '만성 궤양성 치은염', '만성 박리성 치은염', '만성 단순 치주염', '만성 복합 치주염', '만성 치관주위염', '만성 타액선염']
